In [ ]:
import os, sys, glob, shutil
import cv2
from scipy.io import loadmat
from h5py import File
import h5py
from PIL import Image
from six.moves import cPickle as pickle

python_version = sys.version_info.major

###########################################################
##########       AFW to JSON Conversion       ##########
###########################################################

abs_path = 'datasets'
afw_path = os.path.join(abs_path, 'AFW')
json_path = os.path.join(abs_path, 'JSON/')
annotations_file = os.path.join(afw_path, 'anno.mat')

def parse(data, debug=False):
    annotations = data[u'anno']
    '''
        annotations[0] : length of data
    '''
    
    print ('Found {0} rows '.format(len(annotations[0])))
    
    n = len(annotations[0])    
    objects = []
    for indx in range(n):
        object_info = {}
        img_info = annotations[1][indx]
    
        # Image filename
        obj = data[annotations[0][indx]]
        object_info['filename'] = ''.join(chr(i) for i in obj[:])
        if debug:
            print ('Processing {0} ({1}/{2})'.format(object_info['filename'], indx + 1, n))
        object_info['objects'] = []
        for face_indx in range(len(data[img_info])):
            
            face_info = {'class_name':'face'}
            
            # bounding box processing
            obj = data[data[img_info][face_indx][0] ]             
            x1, y1 = int(round(obj[0,0])), int(round(obj[1,0]))
            x2, y2 = int(round(obj[0,1])), int(round(obj[1,1]))
            face_info['bounding_box'] = [x1, y1, x2, y2]            
            
            # pose reading
            obj = data[data[annotations[2][indx]][face_indx][0]]
            yaw, pitch, roll   = float(obj[0]), float(obj[1]), float(obj[2])     
            face_info['pose'] = [yaw, pitch, roll]
            
            object_info['objects'].append(face_info)
            
        objects.append(object_info)
        
    print ('Done.')
    return objects

def make_json_directories():
    """
    Definition: Make directories for json images and annotations.
        Removes previously created json image and annotation directories.
    Returns: None
    """
    if os.path.exists(json_path):
        if python_version == 3:
            prompt = input('Directory already exists. Overwrite? (yes, no): ')
        else:
            prompt = raw_input('Directory already exists. Overwrite? (yes, no): ')
        if prompt == 'no':
            exit(0)
        shutil.rmtree(json_path)
    os.makedirs(json_path)
    os.makedirs(json_path + "images")
    os.makedirs(json_path + "annotations")

def populate_annotations_dir():
    with h5py.File(annotations_file) as f:
        objects = parse (f)
        for i in objects:
            f= open(json_path+'annotations/'+i["filename"].split('.')[0]+".json","w+")
            f.write(str(i))
            f.close()
        
def copy_images_json():
    """
    Definition: Copy all images from afw to training folder in json format.
    """
    for filename in glob.glob(os.path.join(afw_path, "*.*")):
        shutil.copy(filename, json_path + "images/")

def json():
    make_json_directories()
    copy_images_json()
    populate_annotations_dir()
json()